## Webscraping

In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  

In [4]:
# using requests library to scrap the html document
url = 'https://realpython.github.io/fake-jobs/'
response = requests.get(url)

In [5]:
url

'https://realpython.github.io/fake-jobs/'

In [6]:
# checking if the import was successful
response.status_code

200

In [7]:
# response.text - used to inspect the text of the document (it is very unorganized.)

In [8]:
soup_job = BeautifulSoup(response.text, features='html.parser') # parses html into something useable

In [9]:
print(soup_job.prettify()) #returns a organized html document for readability

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Fake Python
  </title>
  <link href="https://cdn.jsdelivr.net/npm/bulma@0.9.2/css/bulma.min.css" rel="stylesheet"/>
 </head>
 <body>
  <section class="section">
   <div class="container mb-5">
    <h1 class="title is-1">
     Fake Python
    </h1>
    <p class="subtitle is-3">
     Fake Jobs for Your Web Scraping Journey
    </p>
   </div>
   <div class="container">
    <div class="columns is-multiline" id="ResultsContainer">
     <div class="column is-half">
      <div class="card">
       <div class="card-content">
        <div class="media">
         <div class="media-left">
          <figure class="image is-48x48">
           <img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
          </figure>
         </div>
         <div class="media-content">
          <h2 c

a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.

In [11]:
soup_job.find('h2')

<h2 class="title is-5">Senior Python Developer</h2>

b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list. 

In [13]:
job_tags = soup_job.findAll('h2')

In [77]:
jobs_list = [job.get_text() for job in job_tags]

c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  

In [16]:
companies = soup_job.findAll('h3', attrs={'class':'company'})

In [17]:
locations = soup_job.findAll('p', attrs={'class':'location'})

In [18]:
posting_dates = soup_job.findAll('p', attrs={'class':'is-small has-text-grey'})

In [19]:
company_list = [company.get_text() for company in companies]

In [20]:
location_list = [location.get_text().strip() for location in locations]

In [21]:
posting_dates_list = [posting_date.get_text().strip('\n') for posting_date in posting_dates]

d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [23]:
job_df = pd.DataFrame(zip(jobs_list, company_list, location_list, posting_dates_list), columns=['Jobs', 'Company', 'location', 'Posting_Dates'])

2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.   

a. First, use the BeautifulSoup find_all method to extract the urls.  

In [26]:
url = []
for link in soup_job.findAll("a", href=True)[1::2]:
    url.append(link['href'])
job_df['url'] = url

b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [87]:
url_obj = ('https://realpython.github.io/fake-jobs/jobs/')+(job_df['Jobs'])
url_obj[0]

'https://realpython.github.io/fake-jobs/jobs/Senior Python Developer'

In [83]:
urls_obj = ('https://realpython.github.io/fake-jobs/jobs/' + (job_df['Jobs'])
    .str.lower()
    .str.replace('[\s/]', '-', regex = True)
    .str.replace('[(),]', '', regex = True)
    .str.replace(' ','-') + '-' + job_df.index.astype('str') + '.html'.format())

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\billy\AppData\Local\Temp\ipykernel_81672\882524825.py:3: SyntaxWarning: invalid escape sequence '\s'
  .str.replace('[\s/]', '-', regex = True)


In [91]:
urls_obj[0]

'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'

3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  

In [33]:
jobdesc_url = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
response = requests.get(jobdesc_url)

In [34]:
soup_jobdesc = BeautifulSoup(response.text, features='html.parser') # parses html into something useable

In [35]:
soup_jobdesc.findAll('p')[1].text

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  

In [37]:
def get_jobdesc(url):
    response = requests.get(url)
    soup_jobdesc = BeautifulSoup(response.text, features='html.parser')
    return soup_jobdesc.findAll('p')[1].text
get_jobdesc("https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html")

'At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.'

c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [39]:
job_desc = job_df['url'].apply(func=get_jobdesc)